<a href="https://colab.research.google.com/github/ch0uchou/Resnet50-MNIST/blob/main/ResNet_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model Resnet50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Preprocess the data
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
# Expand the dimensions of the data to (28, 28, 1) to fit the ConvNet
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)
# Define the identity block
def identity_block(X, f, filters, stage, block):
    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value
    X_shortcut = X
    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name='res' + str(stage) + block + '_branch2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2a')(X)
    X = Activation('relu')(X)
    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               name='res' + str(stage) + block + '_branch2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2b')(X)
    X = Activation('relu')(X)
    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name='res' + str(stage) + block + '_branch2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2c')(X)
    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X
# Define the convolutional block
def convolutional_block(X, f, filters, stage, block, s=2):
    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value
    X_shortcut = X
    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), padding='valid',
               name='res' + str(stage) + block + '_branch2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2a')(X)
    X = Activation('relu')(X)
    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same',
               name='res' + str(stage) + block + '_branch2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2b')(X)
    X = Activation('relu')(X)
    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid',
               name='res' + str(stage) + block + '_branch2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2c')(X)
    # Shortcut Path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), padding='valid',
                        name='res' + str(stage) + block + '_branch1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch1')(X_shortcut)
    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X
# Define the ResNet50 model adjusted for the MNIST dataset
def ResNet50(input_shape=(28, 28, 1), classes=10):
    # Define the input tensor
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(1, 1), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(1, 1))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL . Use "X = AveragePooling2D(...)(X)"
    X = GlobalAveragePooling2D()(X)

    # output layer
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model
# Instantiate the model
model = ResNet50(input_shape=(28, 28, 1), classes=10)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Fit the model on the data
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
# Save the model
model.save('mnist_resnet50.h5')
# Output to indicate save completion
print("Model saved as mnist_resnet50.h5")


11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10
1875/1875 [==============================] - 241s 107ms/step - loss: 0.1837 - accuracy: 0.9518 - val_loss: 0.1388 - val_accuracy: 0.9583
Epoch 2/10
1875/1875 [==============================] - 200s 107ms/step - loss: 0.0771 - accuracy: 0.9781 - val_loss: 0.0444 - val_accuracy: 0.9865
Epoch 3/10
1875/1875 [==============================] - 198s 106ms/step - loss: 0.0548 - accuracy: 0.9836 - val_loss: 0.0923 - val_accuracy: 0.9751
Epoch 4/10
1875/1875 [==============================] - 200s 107ms/step - loss: 0.0481 - accuracy: 0.9855 - val_loss: 0.0625 - val_accuracy: 0.9807
Epoch 5/10
1875/1875 [==============================] - 200s 107ms/step - loss: 0.0419 - accuracy: 0.9870 - val_loss: 0.0717 - val_accuracy: 0.9802
Epoch 6/10
1875/1875 [==============================] - 200s 107ms/step - loss: 0.0358 - accuracy: 0.9889 - val_loss: 0.0256 - val_accuracy: 0.9926
Epoch 7/10
1875/1875 [=========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as mnist_resnet50.h5


In [6]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
# Load the model
model = load_model('mnist_resnet50.h5')
# Load the image
image_path = 'image.png'
image = load_img(image_path, color_mode='grayscale' if model.input_shape[-1] == 1 else 'rgb', target_size=(28, 28))
image = img_to_array(image)
image = image.astype('float32')
image /= 255.0
# MNIST images are usually grayscale; check your model's expected input shape
if model.input_shape[-1] == 1 and image.shape[-1] == 3:
    image = image.mean(axis=-1, keepdims=True)
# Add batch dimension (model expects (batch_size, height, width, channels))
image = np.expand_dims(image, axis=0)
# Predict
prediction = model.predict(image)
# Output prediction
predicted_class = np.argmax(prediction, axis=1)
print(f'Predicted class: {predicted_class[0]}')


1/1 [==============================] - 1s 1s/step
Predicted class: 4
